## El conjunto de datos CelebA 

Utilizaremos el conjunto de datos CelebFaces Attributes para entrenar nuestro modelo autocodificador variacional. Se puede encontrar en Kaggle, [CelebA](https://www.kaggle.com/datasets/jessicali9530/celeba-dataset)

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

print(f'Versión de TensorFlow: {tf.__version__}')
print(f"Dispositivo de entrenamiento: {tf.config.list_physical_devices('GPU')}")

Versión de TensorFlow: 2.10.0
Dispositivo de entrenamiento: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(
    "../datasets/celebA/img",
    labels=None,
    color_mode='rgb',
    image_size=(64, 64),
    batch_size=128,
    shuffle=True,
    seed=42,
    interpolation='bilinear'
)

In [ ]:
def preprocess(img):
    return tf.cast(img, "float32") / 255.0

train = train_data.map(lambda x: preprocess(x))

In [ ]:
class VAE(models.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def call(self, inputs):
        """Call the model on a particular input."""
        z_mean, z_log_var, z = encoder(inputs)
        reconstruction = decoder(z)
        return z_mean, z_log_var, reconstruction

    def train_step(self, data):
        """Step run during training."""
        with tf.GradientTape() as tape:
            z_mean, z_log_var, reconstruction = self(data, training=True)
            reconstruction_loss = tf.reduce_mean(
                BETA * losses.mean_squared_error(data, reconstruction)
            )
            kl_loss = tf.reduce_mean(
                tf.reduce_sum(
                    -0.5
                    * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
                    axis=1,
                )
            )
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        """Step run during validation."""
        if isinstance(data, tuple):
            data = data[0]

        z_mean, z_log_var, reconstruction = self(data)
        reconstruction_loss = tf.reduce_mean(
            BETA * losses.mean_squared_error(data, reconstruction)
        )
        kl_loss = tf.reduce_mean(
            tf.reduce_sum(
                -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
                axis=1,
            )
        )
        total_loss = reconstruction_loss + kl_loss

        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

In [ ]:
vae = VAE(encoder, decoder)
zer = optimizers.Adam(learning_rate=0.0005)
vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))